In [ ]:
from tensorflow.nn import softmax

class Sigmoid:
    
    def __init__(self):
        pass
    def __call__(self, x):
        return (1 / (1 + tf.exp(- x)))

class ReLU:
    def __init__(self):
        pass
    def __call__(self, x):
        return (tf.maximum(0, x))

class SoftMax:
    def __init__(self):
        pass
    def __call__(self, x):
        return (softmax(x).numpy())

class Linear:
    def __init__(self):
        pass
    def __call__(self, x):
        return (x)
    
def choose_activation(name : str):

    if (name == "sigmoid"):
        Activation = Sigmoid()

    elif (name == "relu"):
        Activation = ReLU()

    elif (name == "softmax"):
        Activation = SoftMax()

    else:
        Activation = Linear()
    
    return (Activation)

In [114]:
import numpy as np
import wget 


In [115]:
from keras.layers import Dense

X = np.zeros(shape=(1, 200))
layer = Dense(units=50)
z = layer(X)

np.array(layer.bias).shape

2023-03-07 16:55:26.629568: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


(50,)

In [116]:
layer.weights[0].shape, layer.weights[1].shape

(TensorShape([200, 50]), TensorShape([50]))

In [117]:
X = np.zeros(shape=(200))
W = np.ones(shape=(200, 50))
b = np.zeros(shape=(50))

T = W + b

Z = np.dot(X, W) + b

T.shape, Z.shape

((200, 50), (50,))

### Layer Perceptron
- init all Layer shape with list
- Create forward Prop
- Create Backward Prop

In [118]:
import tensorflow as tf
import numpy as np
from keras.initializers import GlorotUniform
import matplotlib.pyplot as plt
import math

def initialize_weight(input_shape : int, output_shape: int):
        """Initialize Layer Weight with Xavier Initialization

        Args:
            input_shape (int): dimension of input
            output_shape (int): dimension of output

        Returns:
            W : Numpy Array Weight of Layer
        """
        limit_value = np.sqrt(6 / (input_shape + output_shape))

        min_value = -limit_value
        max_value = limit_value
        
        W = np.random.uniform(low =min_value,
                              high=max_value,
                              size=(input_shape, output_shape))
        W = W.astype(np.float32)

        return (W)
    
def initialize_bias(weight_output_shape : int):
    """Initialize the Bias Vector with Zeros

    Args:
        weight_output_shape  (int): output dimension of Weight Matrix

    Returns:
        b : Numpy Array of Zeros
    """
    b = np.zeros(shape=(weight_output_shape))

    return (b)

In [120]:
class Layer:
    
        def __init__(self, input_shape : int, units :int, activation : str):
            
            self.input_shape = input_shape
            self.output_shape = units
            self.W = initialize_weight(input_shape, units)
            self.b = initialize_bias(weight_output_shape=self.W.shape[1])
            self.activation = choose_activation(activation)
            self.A = None
            self.Z = None
            
            self.grads = dict()
            self.grads["dA_next"] = None
            self.grads["dZ"] = None
            self.grads["dW"] = None
            self.grads["db"] = None
        
        def __call__(self, X):
            self.Z = np.dot(X, self.W) + self.b
            self.A = self.activation(self.Z)
            return (self.A)

In [134]:
network_config = [
    Layer(input_shape=200, units=50, activation="relu"),
    Layer(input_shape=50, units=100, activation="relu"),
    Layer(input_shape=100, units=150, activation="relu"),
    Layer(input_shape=150, units=10, activation="softmax"),
]

for layer in network_config:
    print(f"Input Shape : {layer.input_shape} Output Shape : {layer.output_shape}")

Input Shape : 200 Output Shape : 50
Input Shape : 50 Output Shape : 100
Input Shape : 100 Output Shape : 150
Input Shape : 150 Output Shape : 10


In [135]:
for layer in network_config:
    print(layer)

### Cost Function

In [163]:
class CategoricalCrossEntropy:
    
    def __init__(self):
        self.epsilon = 1e-15

    def __call__(self, y_true, y_pred):
        
        self.y_true = y_true
        self.y_pred = y_pred
        
        y_pred_log = np.log(y_pred + self.epsilon)
        class_error = -np.sum(y_true * y_pred_log, axis=1)
        return (class_error)
    
    def backward(self):
        
        dY = self.y_pred + self.epsilon
        dYTrue = - self.y_true / dY
        DLoss = dYTrue / self.y_pred
        
        return (DLoss)

In [164]:
class MeanAbsoluteError:
    
    def __init__(self):
        self.epsilon = 1e-15
    
    def __call__(self, y_true, y_pred):
        
        self.y_true = y_true
        self.y_pred = y_pred
        
        error = np.mean(np.abs(y_pred - y_true))
        
        return (error)
    
    def backward(self):
        
        batch_size = self.y_true.shape[0]
        
        diff = self.y_true - self.y_pred
        
        grad = np.zeros_like(diff)
        grad[diff > 0] = -1
        grad[diff < 0] = 1
        
        random_grad = np.random.uniform(low=-1, high=1, size=np.count_nonzero(diff == 0))
        grad[diff == 0] = random_grad
        
        grad /= batch_size
        
        return (grad)
        

In [165]:
from tabulate import tabulate

class MultiLayerPerceptron:

        def __init__(self, input_shape: int, network_config : list, loss_function):

                self.input_shape = input_shape
                self.network = network_config
                self.loss_function = loss_function
                self.lastActivation = None
        
        def summary(self):
                print("Model")
                print("===============================================")
                
                total_params = 0
                table = list()
                headers=['Name', 'Output Shape', 'Number Params']

                input_layer = ["Input Layer", f"(None, {self.input_shape})", "0"]
                table.append(input_layer)
                
                for index, layer in enumerate(self.network):
                        
                        if index == len(self.network) - 1:
                                name = "Output Layer"
                        else :
                                name = f"layer {index + 1}"

                        output_shape = f"(None, {layer.output_shape})"
                        nbr_params = (layer.W.shape[0] * layer.W.shape[1])

                        element = [name, output_shape, f"{nbr_params:,}"]
                        
                        total_params += nbr_params
                        
                        table.append(element)
                        
                print(tabulate(table, headers))
                print("===============================================")
                print(f"Total Params : {total_params:,}")
                
        def forward(self, input_data):
                
                outputs = list()
                for index, layer in enumerate(self.network):
                        
                        if index == 0:
                                X = input_data
                        else :
                                last_layer = self.network[index - 1]
                                last_activation = last_layer.A
                                X = last_activation
                        
                        output = layer(X)
                        
                self.lastActivation = output
                
                return (self.lastActivation)
        
        def backward(self, y):
                
                ### Last Activation
                # Initializing Backprop with Loss Backward
                Dloss = self.loss_function.backward()
                
                
                return (y)
        
        # def fit(self):
                        
        


In [166]:
X = np.random.uniform(size=(1, 200))

model = MultiLayerPerceptron(input_shape=X.shape[1],
                             network_config=network_config,
                             loss_function=CategoricalCrossEntropy)

In [167]:
loss_function = CategoricalCrossEntropy()

y_hat = model.forward(input_data=X)
y_true = np.array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

loss = loss_function(y_true=y_true, y_pred=y_hat)

loss, loss_function.backward()

(array([2.30258509]),
 array([[-100.,   -0.,   -0.,   -0.,   -0.,   -0.,   -0.,   -0.,   -0.,
           -0.]]))

In [128]:
model.summary()

Model
Name          Output Shape    Number Params
------------  --------------  ---------------
Input Layer   (None, 200)     0
layer 1       (None, 50)      10,000
layer 2       (None, 100)     5,000
layer 3       (None, 150)     15,000
Output Layer  (None, 10)      1,500
Total Params : 31,500


In [106]:
activations = model.forward(X)

for a in activations:
        print(a.shape)

(10,)
